# Import

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 30) 

# Data Load

In [2]:
demo = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/LPOINT_BIG_COMP_01_DEMO.csv')
pdde = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/LPOINT_BIG_COMP_02_PDDE.csv')
cop_u = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/LPOINT_BIG_COMP_03_COP_U.csv')
pd_clac = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/LPOINT_BIG_COMP_04_PD_CLAC.csv')
br = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/LPOINT_BIG_COMP_05_BR.csv')
lpay = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/LPOINT_BIG_COMP_06_LPAY.csv')

Atmoshphere_df = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/Atmoshphere_df.csv')
Weather_df = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/Weather_df.csv')
Holiday_df = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/Holiday.csv',encoding='cp949')

C:\Users\pc\AppData\Local\Temp\ipykernel_5892\1111429996.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  pdde = pd.read_csv('../3.로티로리_데이터 및 모델 세이브 파일/LPOINT_BIG_COMP_02_PDDE.csv')


# Data Preprocess & Check

In [3]:
# 외부 데이터 preprocess
Atmoshphere_df = Atmoshphere_df.iloc[:,1:]
Weather_df = Weather_df.iloc[:,1:]

Weather_df.rename(columns = {'region': 'zon_hlv_store', 
                               'date': 'de_dt'}, inplace=True)

Atmoshphere_df.rename(columns = {'region': 'zon_hlv_store', 
                               'date': 'de_dt'}, inplace=True)

In [4]:
print('1번 데모 데이터 크기: ', demo.shape)
print('2번 유통사구매 데이터 크기: ', pdde.shape)
print('3번 유통사 제외 제휴사 데이터 크기: ', cop_u.shape)
print('4번 물품품목 데이터 크기: ', pd_clac.shape)
print('5번 점포구분 데이터 크기: ', br.shape)
print('6번 엘페이 데이터 크기: ', lpay.shape)

1번 데모 데이터 크기:  (29913, 4)
2번 유통사구매 데이터 크기:  (4381743, 10)
3번 유통사 제외 제휴사 데이터 크기:  (248304, 9)
4번 물품품목 데이터 크기:  (1933, 4)
5번 점포구분 데이터 크기:  (8808, 4)
6번 엘페이 데이터 크기:  (353184, 7)


# Data Merge

### 유통사 구매DF

In [5]:
# 유통사관련 정보 merge
def make_merged_store(pdde, pd_clac, demo, br):
    # 유통사 구매데이터와 상품 정보 merge
    store_info = pd.merge(pdde, pd_clac, on = 'pd_c')

    # 해당 정보와 점포정보 merge
    ## cop_c 정보 겹쳐서 하나 제거
    store_info = pd.merge(store_info ,br.drop('cop_c', axis = 1) ,how = 'left', on='br_c')

    # 해당 정보와 cust 정보 merge
    ## zon_hlv가 겹치니 suffix 생성
    store_info = pd.merge(store_info, demo, on='cust', suffixes =('_store', '_cust'))
    
    return store_info

store_info = make_merged_store(pdde, pd_clac, demo, br)
store_info.head()

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm,zon_hlv_store,zon_mcls,ma_fem_dv,ages,zon_hlv_cust
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,남성티셔츠,남성의류,남성의류상의,Z10,Z10042,여성,50대,Z17
1,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,남성의류,남성의류상의,Z10,Z10042,여성,50대,Z17
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,남성의류,남성의류상의,Z10,Z10042,여성,50대,Z17
3,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,남성의류,남성의류상의,Z10,Z10042,여성,50대,Z17
4,M430112881,A01000005297,1,A01,A010039,PD1692,20210101,10,9900.0,1,커피/음료,테넌트/음식점,식당,Z10,Z10042,여성,50대,Z17


In [6]:
store_info['br_c'].fillna('점포없음', inplace=True)
store_info['zon_hlv_store'].fillna('점포없음', inplace=True)
store_info['zon_mcls'].fillna('점포없음', inplace=True)

### 제휴사 구매DF

In [7]:
# 제휴사 관련 정보 merge
def make_merged_cop(cop_u, demo, br):
    cop_info = pd.merge(cop_u, demo, on = 'cust')
    cop_info = pd.merge(cop_info, br.drop('cop_c', axis=1).rename(columns={'zon_hlv': 'zon_hlv_cop',
                                           'zon_mcls': 'zon_mcls_cop'}),
                        on = 'br_c', how = 'left')

    cop_info.rename({'zon_hlv': 'zon_hlv_cust'}, inplace = True)
    return cop_info

cop_info = make_merged_cop(cop_u, demo, br)
cop_info.head()

,cust,rct_no,cop_c,br_c,chnl_dv,de_dt,vst_dt,de_hr,buy_am,ma_fem_dv,ages,zon_hlv,zon_hlv_cop,zon_mcls_cop
0,M839993508,21102612B015763935,B01,B010012,1,20211026,20211026,12,60000,남성,70대,Z17,Z17,Z17024
1,M839993508,21110610B014219744,B01,B010012,1,20211106,20211106,10,17100,남성,70대,Z17,Z17,Z17024
2,M839993508,21021112B013419710,B01,B010012,1,20210211,20210211,12,136500,남성,70대,Z17,Z17,Z17024
3,M839993508,21092010B012637545,B01,B010012,1,20210920,20210920,10,34200,남성,70대,Z17,Z17,Z17024
4,M839993508,21101009D015920171,D01,D010614,1,20211010,20211010,9,2500,남성,70대,Z17,Z17,Z17018


In [8]:
cop_info['br_c'].fillna('점포없음', inplace = True)
cop_info['zon_hlv_cop'].fillna('점포없음', inplace = True)
cop_info['zon_mcls_cop'].fillna('점포없음', inplace = True)

### Lpay DF

In [9]:
# lpay 관련 정보 merge
def make_merged_lpay(lpay, demo):
    lpay_info = pd.merge(lpay, demo)
    
    return lpay_info

lpay_info = make_merged_lpay(lpay, demo)
lpay_info.head()

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am,ma_fem_dv,ages,zon_hlv
0,M629656521,210803210311226,A03,1,20210803,21,10900,남성,40대,Z04
1,M216016456,210803130167542,L01,2,20210803,13,6860,여성,40대,Z04
2,M216016456,211223110193654,L01,2,20211223,11,12650,여성,40대,Z04
3,M216016456,211223120196103,L01,2,20211223,12,310,여성,40대,Z04
4,M216016456,210401010115485,L01,2,20210401,1,1500,여성,40대,Z04


### 이용고객

In [10]:
print('유통사 이용 고객 수: ', len(set(store_info.cust)))
print('제휴사 이용 고객 수: ',len(set(cop_info.cust)))
print('엘페이 이용 고객 수: ', len(set(lpay.cust)))

유통사 이용 고객 수:  26917
제휴사 이용 고객 수:  21769
엘페이 이용 고객 수:  8906


In [11]:
# 분석에 사용할 데이터프레임 명명

def define_dataframe(store_info, cop_info, lpay_info):
    tp_df = pd.merge(store_info.groupby('cust')['buy_ct'].mean().reset_index(),cop_info.groupby('cust')['buy_am'].mean().reset_index(), on = 'cust', how = 'inner' )
    experience_cust_lst = pd.merge(store_info.groupby('cust')['buy_ct'].mean().reset_index(),cop_info.groupby('cust')['buy_am'].mean().reset_index(), on = 'cust', how = 'inner' )['cust'].tolist()
    experience_lpay_cust_lst = pd.merge(tp_df, lpay_info.groupby('cust')['de_hr'].mean().reset_index(), on = 'cust', how = 'inner')['cust'].tolist()
    
    store_df = store_info[store_info['cust'].isin(experience_cust_lst)].reset_index().iloc[:,1:]
    cop_df = cop_info[cop_info['cust'].isin(experience_cust_lst)].reset_index().iloc[:,1:]
    lpay_df = lpay_info[lpay_info['cust'].isin(experience_lpay_cust_lst)].reset_index().iloc[:,1:]
    lpay_df = lpay_df[(lpay_df['cop_c'] != 'L01')&(lpay_df['cop_c'] != 'L00')]
    
    experience_df = pd.concat([store_df,cop_df], axis = 0)       # 제휴사와 유통사 데이터를 concat한 데이터 프레임
    experience_df = experience_df.drop('zon_hlv', axis = 1)
    
    return experience_df, lpay_df, experience_cust_lst, experience_lpay_cust_lst

master_df, lpay_df, experience_cust_lst, experience_lpay_cust_lst = define_dataframe(store_info, cop_info, lpay_info)

In [12]:
print('온/오프라인 제휴사 및 유통사를 이용하는 고객 수: ', len(experience_cust_lst))
print('제휴사 및 유통사를 이용하면서 엘페이를 이용한 고객 수: ', len(experience_lpay_cust_lst))

온/오프라인 제휴사 및 유통사를 이용하는 고객 수:  18930
제휴사 및 유통사를 이용하면서 엘페이를 이용한 고객 수:  7879


- 유통사와 제휴사를 모두 이용한 고객 중 lpay를 사용한 고객 / 안한고객에 대한 리스트를 구함

### 외부데이터 병합


#### 주말/공휴일 데이터 병합

In [13]:
holiday_list = Holiday_df.Date.unique().tolist()
master_holiday = []
for i in master_df.de_dt:
    if i in holiday_list:
        master_holiday.append(1)
    else:
        master_holiday.append(0)
        
master_df['weekend_holiday'] = master_holiday

#### 날씨정보 병합

In [14]:
def merging_weather_df(master_df):
    # cop랑 store 따로 해야해서 일단 떼놓기
    tp_cop = master_df.iloc[3558050: ,:]
    tp_store = master_df.iloc[:3558050 ,:]

    # 온라인에서 산 녀석들은 점포정보가 없어서 cust 주소로 붙여야 함
    # 근데 한번에 붙이기 위해서 다음과 같이 cust 주소를 store 주소로 임시 대체
    tp_values = tp_store[tp_store['zon_hlv_store'] == '점포없음']['zon_hlv_cust'].values
    tp_store.loc[tp_store['zon_hlv_store'] == '점포없음', 'zon_hlv_store'] = tp_values
    # store 정보와 weather 병합
    tp_store = pd.merge(tp_store , Weather_df, on=['zon_hlv_store', 'de_dt'], how ='left')

    # 온라인에 대한 점포정보가 없어서 가져와주기
    zon_need_cust_lst = list(tp_cop.loc[(tp_cop.zon_hlv_cop == '점포없음')].cust)
    cust_zon_info_lst = tp_store[tp_store['cust'].isin(zon_need_cust_lst)].groupby('cust').zon_hlv_cust.first()
    cust_zon_info_lst_2 = pd.merge(tp_cop.loc[(tp_cop.zon_hlv_cop == '점포없음')],pd.DataFrame(tp_store[tp_store['cust'].isin(zon_need_cust_lst)].groupby('cust').zon_hlv_cust.first()).reset_index() , on = 'cust',
        how = 'left')['zon_hlv_cust_y'].values
    
    # 점포 없음에다 정보 채우기
    tp_cop.loc[(tp_cop.zon_hlv_cop == '점포없음'), 'zon_hlv_cop'] = cust_zon_info_lst_2
    Weather_df.rename(columns = {'zon_hlv_store': 'zon_hlv_cop'}, inplace=True)
    # 정보 임시 cop에 머징
    tp_cop = pd.merge(tp_cop, Weather_df, on=['zon_hlv_cop', 'de_dt'], how ='left')
    
    #2020년 정보가 없어 채워지지 않았기 때문에 20210101로 전부 대체
#     temper = pd.merge(tp_cop[tp_cop.temperature.isna()], Weather_df[Weather_df.de_dt == 20210101], on ='zon_hlv_cop', how = 'left')['temperature_y'].values
#     rainfall = pd.merge(tp_cop[tp_cop.rainfall.isna()], Weather_df[Weather_df.de_dt == 20210101], on ='zon_hlv_cop', how = 'left')['rainfall_y'].values
#     ht = pd.merge(tp_cop[tp_cop.huminity.isna()], Weather_df[Weather_df.de_dt == 20210101], on ='zon_hlv_cop', how = 'left')['huminity_y'].values
#     tp_cop.loc[tp_cop.temperature.isna(), 'temperature'] = temper
#     tp_cop.loc[tp_cop.rainfall.isna(), 'rainfall'] = rainfall
#     tp_cop.loc[tp_cop.huminity.isna(), 'huminity'] = ht
    
    # 다시 master_df로 리턴
    master_df = pd.concat([tp_store, tp_cop], axis = 0)
    return master_df

master_df = merging_weather_df(master_df)

#### 대기정보 병합

In [15]:
# 기상청 정보기준으로 중부 남부 제주 나누기
def merging_atmosphere_df(master_df):
    def change_zone_to_area(x):
        if x == 'Z03'or x == 'Z05'or x == 'Z06'or x == 'Z11'or x == 'Z12'or x == 'Z13'or x == 'Z14'or x == 'Z16':
            return '남부'
        elif x == 'Z07':
            return '제주'
        elif x == 'Z01'or x == 'Z02'or x == 'Z04'or x == 'Z08'or x == 'Z09'or x == 'Z10'or x == 'Z15'or x == 'Z17' :
            return '중부'

    # cop랑 store 따로 해야해서 일단 떼놓기
    tp_cop = master_df.iloc[3558050: ,:]
    tp_store = master_df.iloc[:3558050 ,:]

    # store 정보 handling
    tp_values = tp_store[tp_store['zon_hlv_store'] == '점포없음']['zon_hlv_cust'].values
    tp_store.loc[tp_store['zon_hlv_store'] == '점포없음', 'zon_hlv_store'] = tp_values
    tp_store.loc[:, 'zon_area'] = tp_store['zon_hlv_store'].apply(change_zone_to_area)

    Atmoshphere_df.rename(columns = {'region_at' : 'zon_area'}, inplace=True)

    # store 정보와 atmos 병합
    tp_store = pd.merge(tp_store , Atmoshphere_df, on=['de_dt', 'zon_area'], how ='left')

    # cop 정보 handling
    zon_need_cust_lst = list(tp_cop[tp_cop.zon_hlv_cop == '점포없음'].cust)
    cust_zon_info_lst = tp_store[tp_store['cust'].isin(zon_need_cust_lst)].groupby('cust').zon_hlv_cust.first()
    cust_zon_info_lst_2 = pd.merge(tp_cop.loc[(tp_cop.zon_hlv_cop == '점포없음')],pd.DataFrame(tp_store[tp_store['cust'].isin(zon_need_cust_lst)].groupby('cust').zon_hlv_cust.first()).reset_index() , on = 'cust',
            how = 'left')['zon_hlv_cust_y'].values
    tp_cop.loc[(tp_cop.zon_hlv_cop == '점포없음'), 'zon_hlv_cop'] = cust_zon_info_lst_2
    tp_cop['zon_area'] = tp_cop.loc[:, 'zon_hlv_cop'].apply(change_zone_to_area)

    # cop 정보와 atmos 병합
    tp_cop = pd.merge(tp_cop , Atmoshphere_df, on=['de_dt', 'zon_area'], how ='left')

    # 다시 master_df로 리턴
    master_df = pd.concat([tp_store, tp_cop], axis = 0)
    return master_df


master_df = merging_atmosphere_df(master_df)

C:\Users\pc\AppData\Local\Temp\ipykernel_5892\3554325095.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tp_store.loc[:, 'zon_area'] = tp_store['zon_hlv_store'].apply(change_zone_to_area)
C:\Users\pc\AppData\Local\Temp\ipykernel_5892\3554325095.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tp_cop['zon_area'] = tp_cop.loc[:, 'zon_hlv_cop'].apply(change_zone_to_area)


In [16]:
# 리셋인덱스
master_df.reset_index(drop='first', inplace = True)

In [17]:
master_df_deploy = master_df.copy()
lpay_df_deploy = lpay_df.reset_index(drop = True)

master_df_deploy.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Master DF.csv')
lpay_df_deploy.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Refine lpay DF.csv')

# Quater Division

In [18]:
# 2020년 데이터 삭제
before_2021_lst =master_df[master_df.de_dt < 20210101].index
master_df.drop(before_2021_lst, axis = 0, inplace = True)
master_df.reset_index(drop='first', inplace = True)

In [19]:
master_quater = []
for i in master_df.de_dt:
    if i < 20210401 and i >= 20210101:
        master_quater.append(1)
    elif i < 20210701:
        master_quater.append(2)
    elif i < 20211001:
        master_quater.append(3)
    else:
        master_quater.append(4)    
master_df['quater'] = master_quater

lpay_quater = []
for j in lpay_df.de_dt:
    if j < 20210401 and i >= 20210101:
        lpay_quater.append(1)
    elif j < 20210701:
        lpay_quater.append(2)
    elif j < 20211001:
        lpay_quater.append(3)
    else:
        lpay_quater.append(4)
lpay_df['quater'] = lpay_quater

# Lpay_use_status

In [20]:
def divide_chnl(x):
    if len(x) == 1:
        if x[0] == 1:
            return 'Lpay_Offline'
        else:
            return 'Lpay_Online'
    else:
        return 'Lpay_On/Offline'

In [21]:
# 1 2 분기 train
master_quater12 = master_df.query('quater == 1 or quater == 2 ').reset_index(drop=True)
lpay_df12 = lpay_df.query('quater == 1 or quater == 2 ').reset_index(drop=True)

lpay_df12 = pd.merge(lpay_df12,  pd.DataFrame(lpay_df12.groupby('cust')['chnl_dv'].unique().apply(divide_chnl)).reset_index().rename(columns = {'chnl_dv': 'lpay_use_status'}),  on='cust')
master_quater12 =  pd.merge(master_quater12, lpay_df12.groupby('cust')['lpay_use_status'].unique().apply(lambda x : x[0]).reset_index(), on = 'cust', how ='left')
master_quater12['lpay_use_status'].fillna('Lpay_X', inplace = True)
master_quater12.fillna('정보없음', inplace = True)

In [22]:
# 3분기 test
master_quater3 = master_df.query('quater == 3').reset_index(drop=True)
lpay_df3 = lpay_df.query('quater == 3').reset_index(drop=True)

lpay_df3 = pd.merge(lpay_df3,  pd.DataFrame(lpay_df3.groupby('cust')['chnl_dv'].unique().apply(divide_chnl)).reset_index().rename(columns = {'chnl_dv': 'lpay_use_status'}),  on='cust')
master_quater3 =  pd.merge(master_quater3, lpay_df3.groupby('cust')['lpay_use_status'].unique().apply(lambda x : x[0]).reset_index(), on = 'cust', how ='left')
master_quater3['lpay_use_status'].fillna('Lpay_X', inplace = True)
master_quater3.fillna('정보없음', inplace = True)

In [23]:
# 2 3분기 train
master_quater23 = master_df.query('quater == 2 or quater == 3 ').reset_index(drop=True)
lpay_df23 = lpay_df.query('quater == 2 or quater == 3 ').reset_index(drop=True)

lpay_df23 = pd.merge(lpay_df23,  pd.DataFrame(lpay_df23.groupby('cust')['chnl_dv'].unique().apply(divide_chnl)).reset_index().rename(columns = {'chnl_dv': 'lpay_use_status'}),  on='cust')
master_quater23 =  pd.merge(master_quater23, lpay_df23.groupby('cust')['lpay_use_status'].unique().apply(lambda x : x[0]).reset_index(), on = 'cust', how ='left')
master_quater23['lpay_use_status'].fillna('Lpay_X', inplace = True)
master_quater23.fillna('정보없음', inplace = True)

In [24]:
# 4분기 test
master_quater4 = master_df.query('quater == 4 ').reset_index(drop=True)
lpay_df4 = lpay_df.query('quater == 4 ').reset_index(drop=True)

lpay_df4 = pd.merge(lpay_df4,  pd.DataFrame(lpay_df4.groupby('cust')['chnl_dv'].unique().apply(divide_chnl)).reset_index().rename(columns = {'chnl_dv': 'lpay_use_status'}),  on='cust')
master_quater4 =  pd.merge(master_quater4, lpay_df4.groupby('cust')['lpay_use_status'].unique().apply(lambda x : x[0]).reset_index(), on = 'cust', how ='left')
master_quater4['lpay_use_status'].fillna('Lpay_X', inplace = True)
master_quater4.fillna('정보없음', inplace = True)

In [25]:
master_quater12.shape

(2025851, 29)

In [26]:
master_quater3.shape

(934068, 29)

In [27]:
master_quater23.shape

(1944753, 29)

In [28]:
master_quater4.shape

(830852, 29)

# DataFrame Deployment

In [29]:
master_quater12.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Master_DF12_tr.csv',index= False)
master_quater3.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Master_DF3_te.csv',index= False)
master_quater23.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Master_DF23_tr.csv',index= False)
master_quater4.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Master_DF4_te.csv',index= False)

In [30]:
lpay_df12.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Lpay_DF12.csv',index= False)
lpay_df3.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Lpay_DF3.csv',index= False)
lpay_df23.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Lpay_DF23.csv',index= False)
lpay_df4.to_csv('../3.로티로리_데이터 및 모델 세이브 파일/Lpay_DF4.csv',index= False)